In [ ]:
pip install satang-pro

# Combine

In [181]:
import pandas as pd
import numpy as np

import hashlib
import hmac
import json
import requests
import datetime
import math

from satang_pro import SatangPro

import websocket


def bitkub(btc_sell_list):
    # sample_place_bid   
#     API CONNECTION
    API_HOST = 'https://api.bitkub.com'
    API_KEY = '0dcfc8f1305a0a14f3ec9342ecb15ae9'
    API_SECRET = b'046299c776c54c81158452d5241e63ee'

    def json_encode(data):
        return json.dumps(data, separators=(',', ':'), sort_keys=True)
    # check server time
    response = requests.get(API_HOST + '/api/servertime')
    ts = int(response.text)
    data = {
        'sym': 'THB_BTC', # The symbol
        'lmt': 10
    }
    response = requests.get(API_HOST + '/api/market/books',data)
    a = response.json()
#     bit-ask datafreame
    ask=a['result']['asks']
    bid=a['result']['bids']
    df_bid = pd.DataFrame(columns=["order id","timestamp","volume","rate","amount"])
    df_ask = pd.DataFrame(columns=["order id","timestamp","volume","rate","amount"])
    for i in range(len(bid)):
        df_bid.loc[i]=bid[i]
        df_ask.loc[i]=ask[i]
    # minimum ask sell rate 
    df_ask=df_ask.sort_values(by='rate', ascending=True)
    df_ask['time'] = df_ask['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    # maximum bid buy rate 
    df_bid=df_bid.sort_values(by='rate', ascending=False)
    df_bid['time'] = df_bid['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x))

#     space
    # https://blog.shrimpy.io/blog/cryptocurrency-trading-101-exchange-market-spread?rq=spread
    # AL(lowest ask price) - BH(highest buy price) = Spread
    # Percent Spread = (Spread / lowest ask price) x 100
    max_bid=df_bid.iloc[0]['rate']
    min_ask=df_ask.iloc[0]['rate']
    spread=((min_ask-max_bid)/min_ask)*100

#     slippage
    # kaiko
    avg_buy_price=0
    best_ask=min_ask
    best_bid=max_bid
    midPrice=(best_ask+best_bid)/2
    sum_buy_price=0
    shrimpy_slippage_list=[]
    kaiko_slippage_list=[]
    bybit_slippage_list=[]
    bk_slippage_list=[]
    for btc_sell_amount in btc_sell_list:
        sum_amount=0
        pi=float(df_ask.iloc[0]['rate'])
        sum_buy_price=0 
        for i in range (len(ask)):
            if btc_sell_amount > sum_amount:
                sum_amount=float(df_ask.iloc[i]['amount'])+sum_amount
                sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['rate'])
                if sum_amount>=btc_sell_amount:
                    pf=float(df_ask.iloc[i]['rate'])
                    index=i+1
                    break
        if sum_amount<btc_sell_amount and i==len(ask)-1 :
            bk_slippage_list.append([-1,-1,-1])
            while(len(bk_slippage_list)!= len(btc_sell_list)) :
                bk_slippage_list.append([-1,-1,-1])
            break
        b_sum_amount=0
        sum_ask_price= 0
        b_pi=float(df_bid.iloc[0]['rate'])
        for i in range (len(bid)):
            if btc_sell_amount > b_sum_amount:
                b_sum_amount=float(df_bid.iloc[i]['amount'])+b_sum_amount
                sum_ask_price= sum_ask_price+float(df_bid.iloc[i]['rate'])
                if b_sum_amount>=btc_sell_amount:
                    b_pf=float(df_bid.iloc[i]['rate'])
                    b_index=i+1
                    break
        if b_sum_amount<btc_sell_amount and i==len(ask)-1:
            break
        avg_buy_price=sum_buy_price/(index)
        bybit_slippage=math.fabs(b_pf-pf)/midPrice
        shrimpy_slippage=(math.fabs(pf-pi)/pi)
        kaiko_slippage=(math.fabs(avg_buy_price-midPrice)/midPrice)

        shrimpy_slippage_list.append(shrimpy_slippage)
        kaiko_slippage_list.append(kaiko_slippage)
        bybit_slippage_list.append(bybit_slippage)

        
#         slippage_list.append(shrimpy_slippage_list)
#         slippage_list.append(kaiko_slippage_list)
#         slippage_list.append(bybit_slippage_list)
        bk_slippage_list.append([shrimpy_slippage,kaiko_slippage,bybit_slippage])
    return [df_ask.iloc[0]['time'],bk_slippage_list,spread]


def huobi_thailand(btc_sell_list):
# API
    API_HOST = 'https://www.huobi.co.th/api'
    API_KEY = 'c7c22f11-feb097ef-a6aa3e21-ur2fg6h2gf'
    API_SECRET = b'298d9c50-bc6f69c5-2942dd26-d94df'

    def json_encode(data):
        return json.dumps(data, separators=(',', ':'), sort_keys=True)
    # check server time
    response = requests.get(API_HOST + '/v1/common/timestamp')
    ts = response.json()
    data = {
        'symbol': 'btcthb', # The symbol
        'type': 'step0' #https://www.reddit.com/r/huobi/comments/9xa5xg/what_is_the_purpose_of_step0_step1_step2_step3/
    }
    # Depth Type: step0, step1, step2, step3, step4, step5（merged depth 0-5）- step0 means doesn’t merge
    # When the user selects “Merged Depth”, the market pending orders within the certain quotation accuracy will be combined and displayed. The merged depth only changes the display mode and does not change the actual order price
    # 24 hours trade summary and best bid/ask for a symbol
    # response = requests.get(API_HOST + '/market/depth',data)
    response = requests.get(API_HOST + '/market/depth',data)
    t=int(ts['data'])/1000
    time=datetime.datetime.fromtimestamp(t)
    a = response.json()
    bid=a['tick']['bids']
    ask= a['tick']['asks']
# BID-ASK DATAFRAME
    df_bid = pd.DataFrame(columns=["price","amount"])
    df_ask = pd.DataFrame(columns=["price","amount"])
    for i in range(len(bid)):
        df_bid.loc[i]=bid[i]
    for i in range(len(ask)):
        df_ask.loc[i]=ask[i]
    df_ask=df_ask.sort_values(by='price', ascending=True)
    df_bid=df_bid.sort_values(by='price', ascending=False)
#spread
    # https://blog.shrimpy.io/blog/cryptocurrency-trading-101-exchange-market-spread?rq=spread
    # AL(lowest ask price) - BH(highest buy price) = Spread
    # Percent Spread = (Spread / lowest ask price) x 100
    max_bid=float(df_bid.iloc[0]['price'])
    min_ask=float(df_ask.iloc[0]['price'])
    spread=((min_ask-max_bid)/min_ask)*100
#slippage
    # kaiko
    avg_buy_price=0
    best_ask=min_ask
    best_bid=max_bid
    midPrice=(best_ask+best_bid)/2
    sum_buy_price=0
    shrimpy_slippage_list=[]
    kaiko_slippage_list=[]
    bybit_slippage_list=[]
    hb_slippage_list=[]
    for btc_sell_amount in btc_sell_list:
        sum_amount=0
        pi=float(df_ask.iloc[0]['price'])
        sum_buy_price=0 
        for i in range (len(ask)):
            if btc_sell_amount > sum_amount:
                sum_amount=float(df_ask.iloc[i]['amount'])+sum_amount
                sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
                if sum_amount>=btc_sell_amount:
                    pf=float(df_ask.iloc[i]['price'])
                    index=i+1
                    break
        if sum_amount<btc_sell_amount and i==len(ask)-1:
                hb_slippage_list.append([-1,-1,-1])
                while(len(hb_slippage_list)!= len(btc_sell_list)) :
                    hb_slippage_list.append([-1,-1,-1])
                break
        b_sum_amount=0
        sum_ask_price= 0
        b_pi=float(df_bid.iloc[0]['price'])
        for i in range (len(bid)):
            if btc_sell_amount > b_sum_amount:
                b_sum_amount=float(df_bid.iloc[i]['amount'])+b_sum_amount
                sum_ask_price= sum_ask_price+float(df_bid.iloc[i]['price'])
                if b_sum_amount>=btc_sell_amount:
                    b_pf=float(df_bid.iloc[i]['price'])
                    b_index=i+1
                    break
        if b_sum_amount<btc_sell_amount and i==len(ask)-1:
                break
        avg_buy_price=sum_buy_price/(index)
        bybit_slippage=math.fabs(b_pf-pf)/midPrice
        shrimpy_slippage=(math.fabs(pf-pi)/pi)
        kaiko_slippage=(math.fabs(avg_buy_price-midPrice)/midPrice)

        shrimpy_slippage_list.append(shrimpy_slippage)
        kaiko_slippage_list.append(kaiko_slippage)
        bybit_slippage_list.append(bybit_slippage)
        
        hb_slippage_list.append([shrimpy_slippage,kaiko_slippage,bybit_slippage])
#         hb_slippage_list.append(shrimpy_slippage_list)
#         hb_slippage_list.append(kaiko_slippage_list)
#         hb_slippage_list.append(bybit_slippage_list)
        

    return [time,hb_slippage_list,spread]


def satang(btc_sell_list):
    API_KEY = "live-2a6c1bd5eb0b4321aaaf26721e997e9f"
    SECRET_KEY = "fc8fa6ef2a9e4949bdf72d38208803657659ff67f2a74486a04a64b0bf1f2e6f"
    sp = SatangPro(api_key=API_KEY, secret_key=SECRET_KEY)
    sp.orders(pair='btc_thb')
    bid=sp.orders(pair='btc_thb')['bid']
    ask =sp.orders(pair='btc_thb')['ask']
    sp.orders(pair='btc_thb')['ask']
# BIT-ASK DATAFRAME
    df_bid = pd.DataFrame(columns=["price","amount"])
    df_ask = pd.DataFrame(columns=["price","amount"])
    for i in range(len(bid)):
        df_bid.loc[i]=list(bid[i].values())
        df_ask.loc[i]=list(ask[i].values())
    df_ask=df_ask.sort_values(by='price', ascending=True)
    df_bid=df_bid.sort_values(by='price', ascending=False)
#spread
    # https://blog.shrimpy.io/blog/cryptocurrency-trading-101-exchange-market-spread?rq=spread
    # AL(lowest ask price) - BH(highest buy price) = Spread
    # Percent Spread = (Spread / lowest ask price) x 100
    max_bid=float(df_bid.iloc[0]['price'])
    min_ask=float(df_ask.iloc[0]['price'])
    spread=((min_ask-max_bid)/min_ask)*100
#slippage
    # kaiko
    avg_buy_price=0
    best_ask=min_ask
    best_bid=max_bid
    midPrice=(best_ask+best_bid)/2
    sum_buy_price=0
    shrimpy_slippage_list=[]
    kaiko_slippage_list=[]
    bybit_slippage_list=[]
    sp_slippage_list=[]
    for btc_sell_amount in btc_sell_list:
        sum_amount=0
        pi=float(df_ask.iloc[0]['price'])
        sum_buy_price=0 
        for i in range (len(ask)):
            if btc_sell_amount > sum_amount:
                sum_amount=float(df_ask.iloc[i]['amount'])+sum_amount
                sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
                if sum_amount>=btc_sell_amount:
                    pf=float(df_ask.iloc[i]['price'])
                    index=i+1
                    break
        if sum_amount<btc_sell_amount and i==len(ask)-1:
                sp_slippage_list.append([-1,-1,-1])
                while(len(sp_slippage_list)!= len(btc_sell_list)) :
                    sp_slippage_list.append([-1,-1,-1])
                break
        b_sum_amount=0
        sum_ask_price= 0
        b_pi=float(df_bid.iloc[0]['price'])
        for i in range (len(bid)):
            if btc_sell_amount > b_sum_amount:
                b_sum_amount=float(df_bid.iloc[i]['amount'])+b_sum_amount
                sum_ask_price= sum_ask_price+float(df_bid.iloc[i]['price'])
                if b_sum_amount>=btc_sell_amount:
                    b_pf=float(df_bid.iloc[i]['price'])
                    b_index=i+1
                    break
        if b_sum_amount<btc_sell_amount and i==len(ask)-1:
                    break
        avg_buy_price=sum_buy_price/(index)
        bybit_slippage=math.fabs(b_pf-pf)/midPrice
        shrimpy_slippage=(math.fabs(pf-pi)/pi)
        kaiko_slippage=(math.fabs(avg_buy_price-midPrice)/midPrice)
        
        shrimpy_slippage_list.append(shrimpy_slippage)
        kaiko_slippage_list.append(kaiko_slippage)
        bybit_slippage_list.append(bybit_slippage)
        
        sp_slippage_list.append([shrimpy_slippage,kaiko_slippage,bybit_slippage])
#         sp_slippage_list.append(shrimpy_slippage_list)
#         sp_slippage_list.append(kaiko_slippage_list)
#         sp_slippage_list.append(bybit_slippage_list)
    return [sp_slippage_list,spread]


def bitazza(btc_sell_list):
# API CONNECTION
    ws = websocket.WebSocket()
    ws.connect("wss://apexapi.bitazza.com/WSGateway/")
    frame ={
        "m":0,
        "i":0,
        "n":"OrderBook",
        "o": ""
    }
    payload ={
        "market_pair": "BTCTHB", #THBBTC error
        "level": 2, #// level 1 or level 2 data
        "depth": 10,
    }
    frame["o"]=json.dumps(payload) 
    ws.send(json.dumps(frame))
    result =  ws.recv()
    ws.close()
    json.loads(result)
    a = json.loads(json.loads(result)["o"])
    t=int(a['timestamp'])/1000
    time=datetime.datetime.fromtimestamp(t)
    bid=a['bids']
    ask= a['asks']
# BID-ASK DATAFRAME
    df_bid = pd.DataFrame(columns=["amount","price"])
    df_ask = pd.DataFrame(columns=["amount","price"])
    for i in range(len(bid)):
        df_bid.loc[i]=bid[i]
    for i in range(len(ask)):
        df_ask.loc[i]=ask[i]
    df_ask=df_ask.sort_values(by='price', ascending=True)
    df_bid=df_bid.sort_values(by='price', ascending=False)
# spread
    # https://blog.shrimpy.io/blog/cryptocurrency-trading-101-exchange-market-spread?rq=spread
    # AL(lowest ask price) - BH(highest buy price) = Spread
    # Percent Spread = (Spread / lowest ask price) x 100
    max_bid=float(df_bid.iloc[0]['price'])
    min_ask=float(df_ask.iloc[0]['price'])
    spread=((min_ask-max_bid)/min_ask)*100
# SLIPPAGE
    # kaiko
    avg_buy_price=0
    best_ask=min_ask
    best_bid=max_bid
    midPrice=(best_ask+best_bid)/2
    sum_buy_price=0
    shrimpy_slippage_list=[]
    kaiko_slippage_list=[]
    bybit_slippage_list=[]
    bz_slippage_list=[]
    for btc_sell_amount in btc_sell_list:
        sum_amount=0
        pi=float(df_ask.iloc[0]['price'])
        sum_buy_price=0 
        for i in range (len(ask)):
            if btc_sell_amount > sum_amount:
                sum_amount=float(df_ask.iloc[i]['amount'])+sum_amount
                sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
                if sum_amount>=btc_sell_amount:
                    pf=float(df_ask.iloc[i]['price'])
                    index=i+1
                    break
        if sum_amount<btc_sell_amount and i==len(ask)-1:
            bz_slippage_list.append([-1,-1,-1])
            while(len(bz_slippage_list)!= len(btc_sell_list)) :
                bz_slippage_list.append([-1,-1,-1])
                
            break
        b_sum_amount=0
        sum_ask_price= 0
        b_pi=float(df_bid.iloc[0]['price'])
        for i in range (len(bid)):
            if btc_sell_amount > b_sum_amount:
                b_sum_amount=float(df_bid.iloc[i]['amount'])+b_sum_amount
                sum_ask_price= sum_ask_price+float(df_bid.iloc[i]['price'])
                if b_sum_amount>=btc_sell_amount:
                    b_pf=float(df_bid.iloc[i]['price'])
                    b_index=i+1
                    break

        if b_sum_amount<btc_sell_amount and i==len(ask)-1:
                    break
        avg_buy_price=sum_buy_price/(index)
        
        bybit_slippage=math.fabs(b_pf-pf)/midPrice
        shrimpy_slippage=(math.fabs(pf-pi)/pi)
        kaiko_slippage=(math.fabs(avg_buy_price-midPrice)/midPrice)

        shrimpy_slippage_list.append(shrimpy_slippage)
        kaiko_slippage_list.append(kaiko_slippage)
        bybit_slippage_list.append(bybit_slippage)
        
        bz_slippage_list.append([shrimpy_slippage,kaiko_slippage,bybit_slippage])
#         bz_slippage_list.append(shrimpy_slippage_list)
#         bz_slippage_list.append(kaiko_slippage_list)
#         bz_slippage_list.append(bybit_slippage_list)
    return [time,bz_slippage_list,spread]

# MAIN-----------------------------------

btc_sell_list=[0.1,0.3,0.5,1,5]
bk_time,bk_slippage,bk_spread=bitkub(btc_sell_list)
hb_time,hb_slippage,hb_spread=huobi_thailand(btc_sell_list)
sp_slippage,sp_spread=satang(btc_sell_list)
bz_time,bz_slippage,bz_spread=bitazza(btc_sell_list)

bk_slp_rows=(np.array(bk_slippage).T).tolist()
hb_slp_rows=(np.array(hb_slippage).T).tolist()
sp_slp_rows=(np.array(sp_slippage).T).tolist()
bz_slp_rows=(np.array(bz_slippage).T).tolist()

slippage_spread_rows=[hb_time,bk_slippage,bk_slp_rows[0],bk_slp_rows[1],bk_slp_rows[2],
                      sp_slippage,sp_slp_rows[0],sp_slp_rows[1],sp_slp_rows[2],
                      hb_slippage,hb_slp_rows[0],hb_slp_rows[1],hb_slp_rows[2],
                      bz_slippage,bz_slp_rows[0],bz_slp_rows[1],bz_slp_rows[2]]

# df_slippage_spread = pd.DataFrame(columns=["date","bk-spread","bk-slp-shrimpy","bk-slp-kaiko","bk-slp-bybit",
#                                            "sp-spread","sp-slp-shrimpy","sp-slp-kaiko","sp-slp-bybit",
#                                            "hb-spread","hb-slp-shrimpy","hb-slp-kaiko","hb-slp-bybit",
#                                            "bz-spread","bz-slp-shrimpy","bz-slp-kaiko","bz-slp-bybit"])
# df_slippage_spread.to_csv("//Users//thunchanok//Downloads//df_slippage_spread.csv", index=False)
df=pd.read_csv("//Users//thunchanok//Downloads//df_slippage_spread.csv")
df.loc[len(df)]=slippage_spread_rows
df.to_csv("//Users//thunchanok//Downloads//df_slippage_spread.csv", index=False)
df

,date,bk-spread,bk-slp-shrimpy,bk-slp-kaiko,bk-slp-bybit,sp-spread,sp-slp-shrimpy,sp-slp-kaiko,sp-slp-bybit,hb-spread,hb-slp-shrimpy,hb-slp-kaiko,hb-slp-bybit,bz-spread,bz-slp-shrimpy,bz-slp-kaiko,bz-slp-bybit
0,2020-09-30 10:57:39.968,"[[0.0, 0.0008495514366359849, 0.00169910287327...","[0.0, 0.0, 0.0, 0.0, -1.0]","[0.0008495514366359849, 0.0008495514366359849,...","[0.0016991028732719698, 0.0016991028732719698,...","[[0.0006311800714013143, 0.0004557178633630940...","[0.0006311800714013143, 0.01741399954036478, 0...","[0.00045571786336309403, 0.005810629232681132,...","[0.0016106298027655868, 0.020646045475133185, ...","[[2.500000064828308e-08, 0.12676056516564177, ...","[2.500000064828308e-08, -1.0, -1.0, -1.0, -1.0]","[0.12676056516564177, -1.0, -1.0, -1.0, -1.0]","[0.25352113033128354, -1.0, -1.0, -1.0, -1.0]","[[0.0049295893150270035, 0.005657662014848919,...","[0.0049295893150270035, 0.0049295893150270035,...","[0.005657662014848919, 0.005657662014848919, 0...","[0.013344299794090388, 0.013344299794090388, 0..."
1,2020-09-30 10:57:39.968000,"[[0.0, 0.0008495514366359849, 0.00169910287327...","[0.0, 0.0, 0.0, 0.0, -1.0]","[0.0008495514366359849, 0.0008495514366359849,...","[0.0016991028732719698, 0.0016991028732719698,...","[[0.0006311800714013143, 0.0004557178633630940...","[0.0006311800714013143, 0.01741399954036478, 0...","[0.00045571786336309403, 0.005810629232681132,...","[0.0016106298027655868, 0.020646045475133185, ...","[[2.500000064828308e-08, 0.12676056516564177, ...","[2.500000064828308e-08, -1.0, -1.0, -1.0, -1.0]","[0.12676056516564177, -1.0, -1.0, -1.0, -1.0]","[0.25352113033128354, -1.0, -1.0, -1.0, -1.0]","[[0.0049295893150270035, 0.005657662014848919,...","[0.0049295893150270035, 0.0049295893150270035,...","[0.005657662014848919, 0.005657662014848919, 0...","[0.013344299794090388, 0.013344299794090388, 0..."
2,2020-09-30 11:39:45.364000,"[[0.0, 0.0010572187828002968, 0.00211443756560...","[0.0, 2.9346771129765366e-08, 2.93467711297653...","[0.0010572187828002968, 0.0010572334716988404,...","[0.0021144375656005935, 0.002114466943397681, ...","[[0.0, 0.007319015322687605, 0.014638030645375...","[0.0, 0.0, 0.0, 2.9069857889712696e-08, -1.0]","[0.007319015322687605, 0.007319015322687605, 0...","[0.01463803064537521, 0.01463803064537521, 0.0...","[[2.500000064828308e-08, 0.12676056516564177, ...","[2.500000064828308e-08, -1.0, -1.0, -1.0, -1.0]","[0.12676056516564177, -1.0, -1.0, -1.0, -1.0]","[0.25352113033128354, -1.0, -1.0, -1.0, -1.0]","[[0.0048020005033543095, 0.00562024987487424, ...","[0.0048020005033543095, 0.004802910356699151, ...","[0.00562024987487424, 0.006421838420299898, 0....","[0.012919850688061224, 0.01294669570741796, 0...."


# all exchange API

In [174]:
def bitkub():
    # sample_place_bid
    import hashlib
    import hmac
    import json
    import requests
    import datetime
    import pandas as pd
    import math

    # API info
    # API_HOST = 'https://api.bitkub.com'
    # API_KEY = 'YOUR API KEY'
    # API_SECRET = b'YOUR API SECRET'
    API_HOST = 'https://api.bitkub.com'
    API_KEY = '0dcfc8f1305a0a14f3ec9342ecb15ae9'
    API_SECRET = b'046299c776c54c81158452d5241e63ee'

    def json_encode(data):
        return json.dumps(data, separators=(',', ':'), sort_keys=True)


    # check server time
    response = requests.get(API_HOST + '/api/servertime')
    ts = int(response.text)
    print('Server time: ' + response.text)


    data = {
    #     'sym': 'THB_BTC',
        'sym': 'THB_BTC', # The symbol
        'lmt': 10
    }


    print('Payload with signature: ' + json_encode(data))
    response = requests.get(API_HOST + '/api/market/books',data)

    print('Response: ' + response.text)

    a = response.json()
    ask=a['result']['asks']
    bid=a['result']['bids']


    df_bid = pd.DataFrame(columns=["order id","timestamp","volume","rate","amount"])
    df_ask = pd.DataFrame(columns=["order id","timestamp","volume","rate","amount"])

    for i in range(len(bid)):
        df_bid.loc[i]=bid[i]
        df_ask.loc[i]=ask[i]



    # minimum ask sell rate 
    df_ask=df_ask.sort_values(by='rate', ascending=True)
    df_ask['time'] = df_ask['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x))

    # maximum bid buy rate 
    df_bid=df_bid.sort_values(by='rate', ascending=False)
    df_bid['time'] = df_bid['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x))

    print("df_bid")
    print(df_bid)
    print("df_ask")
    print(df_ask)

    # space
    # https://blog.shrimpy.io/blog/cryptocurrency-trading-101-exchange-market-spread?rq=spread
    # AL(lowest ask price) - BH(highest buy price) = Spread
    # Percent Spread = (Spread / lowest ask price) x 100
    max_bid=df_bid.iloc[0]['rate']
    min_ask=df_ask.iloc[0]['rate']
    spread=((min_ask-max_bid)/min_ask)*100
    print("spread : "+str(spread)+" %")


    # slippage
    # kaiko
    avg_buy_price=0
    best_ask=min_ask
    best_bid=max_bid
    midPrice=(best_ask+best_bid)/2
    sum_buy_price=0
    print("  ")
    print("best_ask "+str(best_ask))
    print("best_bid "+str(best_bid))
    print("midPrice "+str(midPrice))
    btc_sell_list=[0.3,0.5,1,5]
    shrimpy_slippage_list=[]
    kaiko_slippage_list=[]
    bybit_slippage_list=[]
    for btc_sell_amount in btc_sell_list:
        sum_amount=0
        pi=float(df_ask.iloc[0]['rate'])
        sum_buy_price=0 
        print("  ")
        print("___________________________________________________________________________________________________")
        print("ASK ")
    #     print("  ")
        for i in range (len(ask)):
            if btc_sell_amount > sum_amount:
                sum_amount=float(df_ask.iloc[i]['amount'])+sum_amount
                sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['rate'])
                print(" <= accumulative amount => "+str(sum_amount)+" ********** <= accumulative price => "+str(sum_buy_price))
                if sum_amount>=btc_sell_amount:
                    pf=float(df_ask.iloc[i]['rate'])
    #                 sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
                    index=i+1
                    print("     (transaction order : "+str(i+1)+")")
                    print("     (ask final price : "+str(pf)+")")

                    break
        if sum_amount<btc_sell_amount and i==len(ask)-1:
                    print("shrimpy_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
                    print("kaiko_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
                    print("___________________________________________________________________________________________________")
                    print("")


                    break
    #     print("  ")
        print("BID ")
    #     print("  ")
        b_sum_amount=0
        sum_ask_price= 0
        b_pi=float(df_bid.iloc[0]['rate'])
        for i in range (len(bid)):
            if btc_sell_amount > b_sum_amount:
                b_sum_amount=float(df_bid.iloc[i]['amount'])+b_sum_amount
                sum_ask_price= sum_ask_price+float(df_bid.iloc[i]['rate'])
                print(" <= accumulative amount => "+str(b_sum_amount)+" ********** <= accumulative price => "+str(sum_ask_price))
                if b_sum_amount>=btc_sell_amount:
                    b_pf=float(df_bid.iloc[i]['rate'])
    #                 sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
                    b_index=i+1
                    print("     (transaction order : "+str(i+1)+")")
                    print("     (bid final price  : "+str(b_pf)+")")

                    break

        if b_sum_amount<btc_sell_amount and i==len(ask)-1:
                print("bybit_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
                print("___________________________________________________________________________________________________")
                print("")




                break
    #     slippage=((pf-pi)/pi)*100

        avg_buy_price=sum_buy_price/(index)

        print("avg_buy_price "+str(avg_buy_price))

        bybit_slippage=math.fabs(b_pf-pf)/midPrice
        shrimpy_slippage=(math.fabs(pf-pi)/pi)
        kaiko_slippage=(math.fabs(avg_buy_price-midPrice)/midPrice)

        shrimpy_slippage_list.append(shrimpy_slippage)
        kaiko_slippage_list.append(kaiko_slippage)
        bybit_slippage_list.append(bybit_slippage)


        print("shrimpy_slippage of "+str(btc_sell_amount)+" BTC --> "+str(shrimpy_slippage)+" ")
        print("kaiko_slippage of "+str(btc_sell_amount)+" BTC --> "+str(kaiko_slippage)+" ")
        print("bybit_slippage of "+str(btc_sell_amount)+" BTC --> "+str(bybit_slippage)+" ")
        print("___________________________________________________________________________________________________")
        print("")
        
        slippage_list=[]
        slippage_list.append(shrimpy_slippage_list)
        slippage_list.append(kaiko_slippage_list)
        slippage_list.append(bybit_slippage_list)
        
        
        


    return [df_ask.iloc[0]['time'],slippage_list,spread]


In [5]:
def huobi_thailand():
    # sample_place_bid
    import hashlib
    import hmac
    import json
    import requests
    import datetime
    import pandas as pd
    import math


    # API info
    # API_HOST = 'https://api.bitkub.com'
    # API_KEY = 'YOUR API KEY'
    # API_SECRET = b'YOUR API SECRET'
    API_HOST = 'https://www.huobi.co.th/api'
    API_KEY = 'c7c22f11-feb097ef-a6aa3e21-ur2fg6h2gf'
    API_SECRET = b'298d9c50-bc6f69c5-2942dd26-d94df'

    def json_encode(data):
        return json.dumps(data, separators=(',', ':'), sort_keys=True)


    # check server time
    response = requests.get(API_HOST + '/v1/common/timestamp')
    ts = response.json()
    print('Server time: ' + response.text)


    data = {
    #     'sym': 'THB_BTC',
        'symbol': 'btcthb', # The symbol
        'type': 'step0' #https://www.reddit.com/r/huobi/comments/9xa5xg/what_is_the_purpose_of_step0_step1_step2_step3/

    }
    # Depth Type: step0, step1, step2, step3, step4, step5（merged depth 0-5）- step0 means doesn’t merge
    # When the user selects “Merged Depth”, the market pending orders within the certain quotation accuracy will be combined and displayed. The merged depth only changes the display mode and does not change the actual order price


    # 24 hours trade summary and best bid/ask for a symbol
    # response = requests.get(API_HOST + '/market/depth',data)

    # response = requests.get(API_HOST + '/market/detail/merged',data)
    response = requests.get(API_HOST + '/market/depth',data)


    # print('Response: ' + response.text)
    print(response.json())
    t=int(ts['data'])/1000
    time=datetime.datetime.fromtimestamp(t)
    print(time)
    print("Date : "+str(time))


    a = response.json()
    # price amount
    bid=a['tick']['bids']
    ask= a['tick']['asks']





    df_bid = pd.DataFrame(columns=["price","amount"])
    df_ask = pd.DataFrame(columns=["price","amount"])

    for i in range(len(bid)):
        df_bid.loc[i]=bid[i]
    for i in range(len(ask)):
        df_ask.loc[i]=ask[i]

    df_ask=df_ask.sort_values(by='price', ascending=True)
    df_bid=df_bid.sort_values(by='price', ascending=False)
    print("df_bid")
    print(df_bid)
    print("  ")
    print("df_ask")
    print(df_ask)

    # spread
    # https://blog.shrimpy.io/blog/cryptocurrency-trading-101-exchange-market-spread?rq=spread
    # AL(lowest ask price) - BH(highest buy price) = Spread
    # Percent Spread = (Spread / lowest ask price) x 100
    max_bid=float(df_bid.iloc[0]['price'])
    print("max_bid : "+str(max_bid))
    min_ask=float(df_ask.iloc[0]['price'])
    print("min_ask : "+str(min_ask))
    spread=((min_ask-max_bid)/min_ask)*100
    print("spread : "+str(spread)+" %")


    # slippage

    # kaiko
    avg_buy_price=0
    best_ask=min_ask
    best_bid=max_bid
    midPrice=(best_ask+best_bid)/2
    sum_buy_price=0
    print("  ")
    print("best_ask "+str(best_ask))
    print("best_bid "+str(best_bid))
    print("midPrice "+str(midPrice))
    btc_sell_list=[0.3,0.5,1,5]
    shrimpy_slippage_list=[]
    kaiko_slippage_list=[]
    bybit_slippage_list=[]
    hb_slippage_list=[]
    
    for btc_sell_amount in btc_sell_list:
        sum_amount=0
        pi=float(df_ask.iloc[0]['price'])
        sum_buy_price=0 
        print("  ")
        print("___________________________________________________________________________________________________")
        print("ASK ")
    #     print("  ")
        for i in range (len(ask)):
            if btc_sell_amount > sum_amount:
                sum_amount=float(df_ask.iloc[i]['amount'])+sum_amount
                sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
                print(" <= accumulative amount => "+str(sum_amount)+" ********** <= accumulative price => "+str(sum_buy_price))
                if sum_amount>=btc_sell_amount:
                    pf=float(df_ask.iloc[i]['price'])
    #                 sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
                    index=i+1
                    print("     (transaction order : "+str(i+1)+")")
                    print("     (ask final price : "+str(pf)+")")

                    break
        if sum_amount<btc_sell_amount and i==len(ask)-1:
                    print("shrimpy_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
                    print("kaiko_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
                    print("___________________________________________________________________________________________________")
                    print("")


                    break
    #     print("  ")
        print("BID ")
    #     print("  ")
        b_sum_amount=0
        sum_ask_price= 0
        b_pi=float(df_bid.iloc[0]['price'])
        for i in range (len(bid)):
            if btc_sell_amount > b_sum_amount:
                b_sum_amount=float(df_bid.iloc[i]['amount'])+b_sum_amount
                sum_ask_price= sum_ask_price+float(df_bid.iloc[i]['price'])
                print(" <= accumulative amount => "+str(b_sum_amount)+" ********** <= accumulative price => "+str(sum_ask_price))
                if b_sum_amount>=btc_sell_amount:
                    b_pf=float(df_bid.iloc[i]['price'])
    #                 sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
                    b_index=i+1
                    print("     (transaction order : "+str(i+1)+")")
                    print("     (bid final price  : "+str(b_pf)+")")

                    break

        if b_sum_amount<btc_sell_amount and i==len(ask)-1:
                print("bybit_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
                print("___________________________________________________________________________________________________")
                print("")




                break
    #     slippage=((pf-pi)/pi)*100

        avg_buy_price=sum_buy_price/(index)

        print("avg_buy_price "+str(avg_buy_price))

        bybit_slippage=math.fabs(b_pf-pf)/midPrice
        shrimpy_slippage=(math.fabs(pf-pi)/pi)
        kaiko_slippage=(math.fabs(avg_buy_price-midPrice)/midPrice)

        shrimpy_slippage_list.append(shrimpy_slippage)
        kaiko_slippage_list.append(kaiko_slippage)
        bybit_slippage_list.append(bybit_slippage)


        print("shrimpy_slippage of "+str(btc_sell_amount)+" BTC --> "+str(shrimpy_slippage)+" ")
        print("kaiko_slippage of "+str(btc_sell_amount)+" BTC --> "+str(kaiko_slippage)+" ")
        print("bybit_slippage of "+str(btc_sell_amount)+" BTC --> "+str(bybit_slippage)+" ")
        print("___________________________________________________________________________________________________")
        print("")
        
     
        hb_slippage_list.append(shrimpy_slippage_list)
        hb_slippage_list.append(kaiko_slippage_list)
        hb_slippage_list.append(bybit_slippage_list)
        

    return [time,hb_slippage_list,spread]


In [10]:
def satang():
    from satang_pro import SatangPro
    API_KEY = "live-2a6c1bd5eb0b4321aaaf26721e997e9f"
    SECRET_KEY = "fc8fa6ef2a9e4949bdf72d38208803657659ff67f2a74486a04a64b0bf1f2e6f"

    sp = SatangPro(api_key=API_KEY, secret_key=SECRET_KEY)
    sp.orders(pair='btc_thb')

    bid=sp.orders(pair='btc_thb')['bid']
    ask =sp.orders(pair='btc_thb')['ask']

    sp.orders(pair='btc_thb')['ask']

    import pandas as pd
    df_bid = pd.DataFrame(columns=["price","amount"])
    df_ask = pd.DataFrame(columns=["price","amount"])

    for i in range(len(bid)):
        df_bid.loc[i]=list(bid[i].values())
        df_ask.loc[i]=list(ask[i].values())

    df_ask=df_ask.sort_values(by='price', ascending=True)
    df_bid=df_bid.sort_values(by='price', ascending=False)
    print("df_bid")
    print(df_bid)
    print("  ")
    print("df_ask")
    print(df_ask)

    # spread
    # https://blog.shrimpy.io/blog/cryptocurrency-trading-101-exchange-market-spread?rq=spread
    # AL(lowest ask price) - BH(highest buy price) = Spread
    # Percent Spread = (Spread / lowest ask price) x 100
    max_bid=float(df_bid.iloc[0]['price'])
    print("max_bid : "+str(max_bid))
    min_ask=float(df_ask.iloc[0]['price'])
    print("min_ask : "+str(min_ask))
    spread=((min_ask-max_bid)/min_ask)*100
    print("spread : "+str(spread)+" %")


    # slippage
    import math
    # kaiko
    avg_buy_price=0
    best_ask=min_ask
    best_bid=max_bid
    midPrice=(best_ask+best_bid)/2
    sum_buy_price=0
    print("  ")
    print("best_ask "+str(best_ask))
    print("best_bid "+str(best_bid))
    print("midPrice "+str(midPrice))
    btc_sell_list=[0.3,0.5,1,5]
    shrimpy_slippage_list=[]
    kaiko_slippage_list=[]
    bybit_slippage_list=[]
    sp_slippage_list=[]
    for btc_sell_amount in btc_sell_list:
        sum_amount=0
        pi=float(df_ask.iloc[0]['price'])
        sum_buy_price=0 
        print("  ")
        print("___________________________________________________________________________________________________")
        print("ASK ")
    #     print("  ")
        for i in range (len(ask)):
            if btc_sell_amount > sum_amount:
                sum_amount=float(df_ask.iloc[i]['amount'])+sum_amount
                sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
                print(" <= accumulative amount => "+str(sum_amount)+" ********** <= accumulative price => "+str(sum_buy_price))
                if sum_amount>=btc_sell_amount:
                    pf=float(df_ask.iloc[i]['price'])
    #                 sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
                    index=i+1
                    print("     (transaction order : "+str(i+1)+")")
                    print("     (ask final price : "+str(pf)+")")

                    break
        if sum_amount<btc_sell_amount and i==len(ask)-1:
                    print("shrimpy_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
                    print("kaiko_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
                    print("___________________________________________________________________________________________________")
                    print("")


                    break
    #     print("  ")
        print("BID ")
    #     print("  ")
        b_sum_amount=0
        sum_ask_price= 0
        b_pi=float(df_bid.iloc[0]['price'])
        for i in range (len(bid)):
            if btc_sell_amount > b_sum_amount:
                b_sum_amount=float(df_bid.iloc[i]['amount'])+b_sum_amount
                sum_ask_price= sum_ask_price+float(df_bid.iloc[i]['price'])
                print(" <= accumulative amount => "+str(b_sum_amount)+" ********** <= accumulative price => "+str(sum_ask_price))
                if b_sum_amount>=btc_sell_amount:
                    b_pf=float(df_bid.iloc[i]['price'])
    #                 sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
                    b_index=i+1
                    print("     (transaction order : "+str(i+1)+")")
                    print("     (bid final price  : "+str(b_pf)+")")

                    break

        if b_sum_amount<btc_sell_amount and i==len(ask)-1:
                print("bybit_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
                print("___________________________________________________________________________________________________")
                print("")




                break
    #     slippage=((pf-pi)/pi)*100

        avg_buy_price=sum_buy_price/(index)

        print("avg_buy_price "+str(avg_buy_price))

        bybit_slippage=math.fabs(b_pf-pf)/midPrice
        shrimpy_slippage=(math.fabs(pf-pi)/pi)
        kaiko_slippage=(math.fabs(avg_buy_price-midPrice)/midPrice)

        shrimpy_slippage_list.append(shrimpy_slippage)
        kaiko_slippage_list.append(kaiko_slippage)
        bybit_slippage_list.append(bybit_slippage)
        
        sp_slippage_list.append(shrimpy_slippage_list)
        sp_slippage_list.append(kaiko_slippage_list)
        sp_slippage_list.append(bybit_slippage_list)


        print("shrimpy_slippage of "+str(btc_sell_amount)+" BTC --> "+str(shrimpy_slippage)+" ")
        print("kaiko_slippage of "+str(btc_sell_amount)+" BTC --> "+str(kaiko_slippage)+" ")
        print("bybit_slippage of "+str(btc_sell_amount)+" BTC --> "+str(bybit_slippage)+" ")
        print("___________________________________________________________________________________________________")
        print("")

    return [sp_slippage_list,spread]


In [16]:
def bitazza():
    import websocket
    import json
    import datetime
    import pandas as pd
    import math

    ws = websocket.WebSocket()
    ws.connect("wss://apexapi.bitazza.com/WSGateway/")

    frame ={
        "m":0,
        "i":0,
        "n":"OrderBook",
        "o": ""
    }
    payload ={
        "market_pair": "BTCTHB", #THBBTC error
        "level": 2, #// level 1 or level 2 data
        "depth": 10,
    }
    frame["o"]=json.dumps(payload) 
    # frame["o"]=payload 
    ws.send(json.dumps(frame))
    result =  ws.recv()

    ws.close()
    json.loads(result)


    a = json.loads(json.loads(result)["o"])


    t=int(a['timestamp'])/1000
    time=datetime.datetime.fromtimestamp(t)
    print (time)
    bid=a['bids']
    ask= a['asks']
    print("-------bid[0] :")
    print(bid[0])
    print("-------ask[0] :")
    print(ask[0])



    df_bid = pd.DataFrame(columns=["amount","price"])
    df_ask = pd.DataFrame(columns=["amount","price"])

    for i in range(len(bid)):
        df_bid.loc[i]=bid[i]
    for i in range(len(ask)):
        df_ask.loc[i]=ask[i]

    df_ask=df_ask.sort_values(by='price', ascending=True)
    df_bid=df_bid.sort_values(by='price', ascending=False)
    print("df_bid")
    print(df_bid)
    print("  ")
    print("df_ask")
    print(df_ask)

    # spread
    # https://blog.shrimpy.io/blog/cryptocurrency-trading-101-exchange-market-spread?rq=spread
    # AL(lowest ask price) - BH(highest buy price) = Spread
    # Percent Spread = (Spread / lowest ask price) x 100
    max_bid=float(df_bid.iloc[0]['price'])
    print("max_bid : "+str(max_bid))
    min_ask=float(df_ask.iloc[0]['price'])
    print("min_ask : "+str(min_ask))
    spread=((min_ask-max_bid)/min_ask)*100
    print("spread : "+str(spread)+" %")


    # kaiko
    avg_buy_price=0
    best_ask=min_ask
    best_bid=max_bid
    midPrice=(best_ask+best_bid)/2
    sum_buy_price=0
    print("  ")
    print("best_ask "+str(best_ask))
    print("best_bid "+str(best_bid))
    print("midPrice "+str(midPrice))
    btc_sell_list=[0.3,0.5,1,5]
    shrimpy_slippage_list=[]
    kaiko_slippage_list=[]
    bybit_slippage_list=[]
    bz_slippage_list=[]
    for btc_sell_amount in btc_sell_list:
        sum_amount=0
        pi=float(df_ask.iloc[0]['price'])
        sum_buy_price=0 
        print("  ")
        print("___________________________________________________________________________________________________")
        print("ASK ")
    #     print("  ")
        for i in range (len(ask)):
            if btc_sell_amount > sum_amount:
                sum_amount=float(df_ask.iloc[i]['amount'])+sum_amount
                sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
                print(" <= accumulative amount => "+str(sum_amount)+" ********** <= accumulative price => "+str(sum_buy_price))
                if sum_amount>=btc_sell_amount:
                    pf=float(df_ask.iloc[i]['price'])
    #                 sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
                    index=i+1
                    print("     (transaction order : "+str(i+1)+")")
                    print("     (ask final price : "+str(pf)+")")

                    break
        if sum_amount<btc_sell_amount and i==len(ask)-1:
                    print("shrimpy_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
                    print("kaiko_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
                    print("___________________________________________________________________________________________________")
                    print("")


                    break
    #     print("  ")
        print("BID ")
    #     print("  ")
        b_sum_amount=0
        sum_ask_price= 0
        b_pi=float(df_bid.iloc[0]['price'])
        for i in range (len(bid)):
            if btc_sell_amount > b_sum_amount:
                b_sum_amount=float(df_bid.iloc[i]['amount'])+b_sum_amount
                sum_ask_price= sum_ask_price+float(df_bid.iloc[i]['price'])
                print(" <= accumulative amount => "+str(b_sum_amount)+" ********** <= accumulative price => "+str(sum_ask_price))
                if b_sum_amount>=btc_sell_amount:
                    b_pf=float(df_bid.iloc[i]['price'])
    #                 sum_buy_price= sum_buy_price+float(df_ask.iloc[i]['price'])
                    b_index=i+1
                    print("     (transaction order : "+str(i+1)+")")
                    print("     (bid final price  : "+str(b_pf)+")")

                    break

        if b_sum_amount<btc_sell_amount and i==len(ask)-1:
                print("bybit_slippage of "+str(btc_sell_amount)+" BTC --> null  --> not enough")
                print("___________________________________________________________________________________________________")
                print("")




                break
    #     slippage=((pf-pi)/pi)*100

        avg_buy_price=sum_buy_price/(index)

        print("avg_buy_price "+str(avg_buy_price))

        bybit_slippage=math.fabs(b_pf-pf)/midPrice
        shrimpy_slippage=(math.fabs(pf-pi)/pi)
        kaiko_slippage=(math.fabs(avg_buy_price-midPrice)/midPrice)

        shrimpy_slippage_list.append(shrimpy_slippage)
        kaiko_slippage_list.append(kaiko_slippage)
        bybit_slippage_list.append(bybit_slippage)
        
        bz_slippage_list.append(shrimpy_slippage_list)
        bz_slippage_list.append(kaiko_slippage_list)
        bz_slippage_list.append(bybit_slippage_list)



        print("shrimpy_slippage of "+str(btc_sell_amount)+" BTC --> "+str(shrimpy_slippage)+" ")
        print("kaiko_slippage of "+str(btc_sell_amount)+" BTC --> "+str(kaiko_slippage)+" ")
        print("bybit_slippage of "+str(btc_sell_amount)+" BTC --> "+str(bybit_slippage)+" ")
        print("___________________________________________________________________________________________________")
        print("")

    return [time,bz_slippage_list,spread]

In [ ]:
bk_time,bk_slippage,bk_spread=bitkub()
hb_time,hb_slippage,hb_spread=huobi_thailand()
sp_slippage,sp_spread=satang()
bz_time,bz_slippage,bz_spread=bitazza()